In [ ]:
from clesto import *

#### Parameters

In [ ]:
n = 3
p = 3

arity = p
degree = (n-1)*(p-1)
torsion = p
convention = 'Berger-Fresse'

#### Using the E-infinity construction

In [ ]:
surj = Surjection.steenrod_product(arity, degree, torsion, convention)
print(f'surj = {surj}.\n')
print(f'complexity = {surj.complexity}.\n')
print(f'd_surj = {surj.boundary()}.\n')
print(f'[d_surj] = {surj.boundary().orbit()}')

#### Turchin representative

In [ ]:
x = Surjection_element({(1,2,1):1}, p, convention)
turchin = x.compose(x, 1)
print(f'{x} o_1 {x} = {turchin}')
print(f'its boundary is {turchin.boundary()}')
print(f'whose twisted orbit is {turchin.boundary().orbit("sign")}')

#### All orbits arising from compositions

In [ ]:
from itertools import product 

comp = []
x = Surjection_element({(2,1,2,1): 1}, p, convention)
rho = SymmetricRing.rotation_element(2, torsion=p)

for a, b, i in product([x, rho * x], [x, rho * x], {1,2}):
    comp.append(a.compose(b, i).orbit())
    
for surj in set(comp):
    print(surj)

#### Salvatore representative

In [ ]:
salvatore = Surjection_element({(1, 2, 1, 2, 3, 2, 3):1,
                                (1, 2, 3, 1, 3, 2, 3):1,
                                (1, 2, 3, 2, 1, 2, 3):1,
                                (1, 2, 1, 3, 1, 2, 3):2},
                               torsion, convention)
print(salvatore)
print(salvatore.boundary().orbit())

#### Suspension

In [ ]:
print(salvatore.suspension().boundary().orbit('sign'))

In [ ]:
from itertools import product

# wild card (1,2,3,2,3,1,3,1,2)

cand = Surjection_element({(1,2,3,2,1,2,3,2,1):1,(1,2,1,3,1,2,3,2,1):-1,(1,2,3,1,3,2,3,2,1):1,(1,2,1,2,3,2,3,2,1):1,

(1,2,3,2,1,3,1,2,1):1,(1,2,3,2,3,1,3,2,1):-1,(1,2,3,2,3,2,1,2,1):1},3,'Berger-Fresse')

keys = [(1,2,1,3,1,3,1,2,1), (1,2,1,3,1,2,1,3,1), (1,2,1,2,1,3,1,3,1), (1,2,1,2,3,2,1,2,1), (1,2,1,3,2,3,1,3,1), (1,2,1,2,3,2,3,1,3), (1,2,1,3,1,3,2,3,2)]
candidates = []
for coeff in product([0,1,2], repeat=7):
    candidate = cand + Surjection_element({k: v for k,v in zip(keys, coeff)}, 3, convention)
    if not candidate.boundary().orbit('sign'):
        print(candidate)
print('finished')